In [1]:
import os
os.chdir("../../")

import numpy as np
import pandas as pd

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from tqdm import tqdm

from src.text.gui import DrawTopics
from src.google_trends import *

In [3]:
data_dir = os.getcwd() + "/data/text/solomon_islands/"
target_dir = os.getcwd() + "/outputs/text/solomon_islands/"

filepaths = [data_dir+file for file in os.listdir(data_dir) if "news" in file]

## GUI

## Sentiment Analysis

In [5]:
sid = SentimentIntensityAnalyzer()
news_df = pd.DataFrame()

for name, file in zip(["sibc", "ss", "st", "tis"], filepaths):
    df = pd.read_csv(file).drop("Unnamed: 0", axis=1)
    df["date"] = pd.to_datetime(df["date"], utc=False)
    score_df = pd.DataFrame()
    with tqdm(total=len(df)) as pbar:
        for news in df.news:
            news = str(news)
            scores = sid.polarity_scores(news)
            ss_temp = pd.DataFrame(scores, columns=scores.keys(), index=[0])
            if score_df.empty:
                score_df = ss_temp
            else:
                score_df = pd.concat([score_df, ss_temp], axis=0)
            pbar.update(1)
    df["score"] = score_df["compound"].tolist()
    df_sent = (df.set_index("date").groupby(pd.Grouper(freq="MS"))[["score"]].mean().reset_index()
                 .rename({"score": name}, axis=1))
    df_sent["date"] = [pd.to_datetime(d.date()) for d in df_sent.date]
    if news_df.empty:
        news_df = df_sent
    else:
        news_df = news_df.merge(df_sent, how="outer", on="date")

100%|██████████████████████████████████████████████████████████████████████████| 9017/9017 [00:17<00:00, 520.88it/s]


In [12]:
news_df = news_df.sort_values(by="date").reset_index(drop=True)
news_df.to_csv(target_dir+"sib_sentiment_monthly.csv", encoding="utf-8")